In [ ]:
# Loading required libraries and functions
import os
import shutil
import random
import itertools
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib as mpl
from keras import backend
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from keras.applications import imagenet_utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, Activation
from sklearn.metrics import precision_score, recall_score
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import decode_predictions, preprocess_input

In [ ]:
labels = ['Flooding', 'No Flooding']
train_path = '/gdrive/MyDrive/Dpl/data_source/new_data/train'
valid_path = '/gdrive/MyDrive/Dpl/data_source/new_data/valid'
test_path = '/gdrive/MyDrive/Dpl/data_source/new_data/test'

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224,224), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=10, shuffle=False)

In [ ]:
#Loading pre-trained lightweight mobilenet image classifier
mobile = tf.keras.applications.mobilenet.MobileNet(weights='imagenet', include_top=False)
# mobile.summary()

In [ ]:
# Store all layers of the original mobilenet except the last 5 layers in variable x
# There is no predefined logic behind this, it just gives the optimal results for this task
# Also, we will be only training the last 12 layers of the mobilenet during finetuning as we want
# it to keep all of the previously learned weights
x = mobile.layers[-12].output
x

In [ ]:
# Create global pooling, dropout and a binary output layer, as we want our model to be a binary classifier,
# i.e. to classify flooding and no flooding
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
output = Dense(units=2, activation='sorfmax')(x)

In [ ]:
# Construct the new fine-tuned mode
model = Model(inputs=mobile.input, outputs=output)

In [ ]:
# Freez weights of all the layers except for the last five layers in our new model,
# meaning that only the last 12 layers of the model will be trained.
for layer in model.layers[:-23]:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x=train_batches,
          steps_per_epoch=len(train_batches),
          validation_data=valid_batches,
          validation_steps=len(valid_batches),
          epochs=10,
          verbose=2
)

In [ ]:
# Saving and loading our trained for future use

model.save("fine_tuned_flood_detection_model")
# model.load_weights('fine_tuned_flood_detection_model')

In [ ]:
# Make predictions and plot confusion matrix to look how well our model performed in classifying
# flooding and no flooding images

test_labels = test_batches.classes
predictions = model.predict(x=test_batches, steps=len(test_batches), verbose=0)
cm = confusion_matrix(y_true=test_labels, y_pred=predictions.argmax(axis=1))
precision = precision_score(y_true=test_labels, y_pred=predictions.argmax(axis=1))
f1_score = f1_score(y_true=test_labels, y_pred=predictions.argmax(axis=1))
accuracy = accuracy_score(y_true=test_labels, y_pred=predictions.argmax(axis=1))
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Pring precision, F1 score and accuracy of our model
print('Precision: ', precision)
print('F1 Score: ', f1_score)
print('Accuracy: ', accuracy)

In [ ]:
# Confusion Matrix
test_batches.class_indices
cm_plot_labels = ['Flooding','No Flooding']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

In [ ]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from IPython.display import display, Image
def preprocess_image(file_path):
    img = image.load_img(file_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)
def predict_on_folder(folder_path, model, labels, output_csv_path):
    results = []

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path) and file_name.endswith(('.jpg', '.png', '.jpeg', '.gif')):
            preprocessed_image = preprocess_image(file_path)
            predictions = model.predict(preprocessed_image)
            result = np.argmax(predictions)
            file_id = os.path.splitext(file_name)[0]  # Lấy phần ID của file (bỏ đuôi)
            results.append({'id': file_id, 'label': labels[result]})

    # Chuyển danh sách kết quả thành DataFrame và lưu thành file CSV
    df_results = pd.DataFrame(results)
    df_results['label'] = df_results['label'].apply(lambda x: 1 if x == 'Flooding' else 0)
    df_results.to_csv(output_csv_path, index=False)

# Đường dẫn tới thư mục chứa ảnh và file CSV đầu ra
img_folder_path = '/gdrive/MyDrive/Dpl/testset_source/new_testset'
output_csv_path = '/gdrive/MyDrive/Dpl/predictions.csv'

# Gọi hàm để thực hiện dự đoán và lưu kết quả
predict_on_folder(img_folder_path, model, labels, output_csv_path)

print(f"Kết quả dự đoán đã được lưu vào {output_csv_path}")